In [1]:
import numpy as np
import pandas as pd
import gc
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras import backend as K
from sklearn.model_selection import KFold

# Metric and Loss Functions

In [2]:
def mcrmse(t, p, seq_len_target):
    t = t[:, :seq_len_target]
    p = p[:, :seq_len_target]
    score = np.mean(np.sqrt(np.mean(np.mean((p - t) ** 2, axis=1), axis=0)))
    return score


def mcrmse_loss(t, y, seq_len_target):
    t = t[:, :seq_len_target]
    y = y[:, :seq_len_target]
    loss = tf.reduce_mean(tf.sqrt(tf.reduce_mean(tf.reduce_mean((t - y) ** 2, axis=1), axis=0)))
    return loss

# Custom Loss Layer for the Autoencoder

In [3]:
class AutoencoderLossLayer(L.Layer):
    def __init__(self, **kwargs):
        super(AutoencoderLossLayer, self).__init__(**kwargs)
    
    def call(self, inputs):
        node, p = inputs
        # Custom loss for denoising autoencoder (binary cross-entropy like)
        loss = -tf.reduce_mean(20 * node * tf.math.log(p + 1e-4) + (1 - node) * tf.math.log(1 - p + 1e-4))
        return loss

# Attention and Model Building Blocks

In [4]:
def attention(x_inner, x_outer, n_factor):
    x_Q = L.Conv1D(n_factor, 1, activation='linear',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='glorot_uniform')(x_inner)
    x_K = L.Conv1D(n_factor, 1, activation='linear',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='glorot_uniform')(x_outer)
    x_V = L.Conv1D(n_factor, 1, activation='linear',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='glorot_uniform')(x_outer)
    x_KT = L.Permute((2, 1))(x_K)
    res = L.Lambda(lambda c: K.batch_dot(c[0], c[1]) / np.sqrt(n_factor))([x_Q, x_KT])
    att = L.Lambda(lambda c: K.softmax(c, axis=-1))(res)
    att = L.Lambda(lambda c: K.batch_dot(c[0], c[1]))([att, x_V])
    return att


def multi_head_attention(x, y, n_factor, n_head, dropout):
    if n_head == 1:
        att = attention(x, y, n_factor)
    else:
        n_factor_head = n_factor // n_head
        heads = [attention(x, y, n_factor_head) for i in range(n_head)]
        att = L.Concatenate()(heads)
        att = L.Dense(n_factor,
                      kernel_initializer='glorot_uniform',
                      bias_initializer='glorot_uniform')(att)
    x = L.Add()([x, att])
    x = L.LayerNormalization()(x)
    if dropout > 0:
        x = L.Dropout(dropout)(x)
    return x


def res(x, unit, kernel=3, rate=0.1):
    h = L.Conv1D(unit, kernel, 1, padding="same", activation=None)(x)
    h = L.LayerNormalization()(h)
    h = L.LeakyReLU()(h)
    h = L.Dropout(rate)(h)
    return L.Add()([x, h])


def forward(x, unit, kernel=3, rate=0.1):
    h = L.Conv1D(unit, kernel, 1, padding="same", activation=None)(x)
    h = L.LayerNormalization()(h)
    h = L.Dropout(rate)(h)
    h = L.LeakyReLU()(h)
    h = res(h, unit, kernel, rate)
    return h


def adj_attn(x, adj, unit, n=2, rate=0.1):
    x_a = x
    x_as = []
    for i in range(n):
        x_a = forward(x_a, unit)
        x_a = L.Lambda(lambda inputs: tf.matmul(inputs[0], inputs[1]))([adj, x_a])
        x_as.append(x_a)
    if n == 1:
        x_a = x_as[0]
    else:
        x_a = L.Concatenate()(x_as)
    x_a = forward(x_a, unit)
    return x_a


def get_base(X_node_shape_2, As_shape_3):
    node = tf.keras.Input(shape=(None, X_node_shape_2), name="node")
    adj = tf.keras.Input(shape=(None, None, As_shape_3), name="adj")

    # Learnable adjacency component
    adj_learned = L.Dense(1, "relu")(adj)
    adj_all = L.Concatenate(axis=3)([adj, adj_learned])

    # Initial feature extraction using multiple Conv1D layers with different kernel sizes
    xs = []
    xs.append(node)
    x1 = forward(node, 128, kernel=3, rate=0.0)
    x2 = forward(x1, 64, kernel=6, rate=0.0)
    x3 = forward(x2, 32, kernel=15, rate=0.0)
    x4 = forward(x3, 16, kernel=30, rate=0.0)
    x = L.Concatenate()([x1, x2, x3, x4])

    # Graph convolution and multi-head attention layers
    for unit in [64, 32]:
        x_as = []
        for i in range(adj_all.shape[3]):
            adj_slice = L.Lambda(lambda inputs, idx=i: inputs[:, :, :, idx])(adj_all)
            x_a = adj_attn(x, adj_slice, unit, rate=0.0)
            x_as.append(x_a)
        x_c = forward(x, unit, kernel=30)
        
        x = L.Concatenate()(x_as + [x_c])
        x = forward(x, unit)
        x = multi_head_attention(x, x, unit, 4, 0.0)
        xs.append(x)
        
    x = L.Concatenate()(xs)

    model = tf.keras.Model(inputs=[node, adj], outputs=x)
    return model


def get_ae_model(base_model, X_node_shape_2):
    node = tf.keras.Input(shape=(None, X_node_shape_2), name="node")
    adj = tf.keras.Input(shape=(None, None, base_model.input[1].shape[3]), name="adj")

    # Apply spatial dropout to the node input for denoising
    x = base_model([L.SpatialDropout1D(0.3)(node), adj])    
    x = L.Dense(64, activation='relu')(x)
    x = L.Dropout(0.3)(x)
    p = L.Dense(X_node_shape_2, activation='sigmoid')(x)

    loss = AutoencoderLossLayer()([node, p])

    model = tf.keras.Model(inputs=[node, adj], outputs=[loss])
    
    opt = tf.optimizers.Adam()
    model.compile(optimizer=opt, loss=lambda t, y: y)
    return model


def get_model(base_model, seq_len_target, As_shape_3):
    node = tf.keras.Input(shape=(None, base_model.input[0].shape[2]), name="node")
    adj = tf.keras.Input(shape=(None, None, As_shape_3), name="adj")
    
    x = base_model([node, adj])
    x = L.Dense(128, activation='relu')(x)
    x = L.Dropout(0.4)(x)
    x = L.Dense(5, activation=None)(x)

    model = tf.keras.Model(inputs=[node, adj], outputs=[x])
    
    opt = tf.optimizers.Adam()
    # Using the custom MCRMSE loss for compilation
    model.compile(optimizer=opt, loss=lambda t, y: mcrmse_loss(t, y, seq_len_target))
    return model

# Data Loading and Preprocessing

In [5]:
# Load datasets
train = pd.read_json("./stanford-covid-vaccine/train.json", lines=True)
test  = pd.read_json("./stanford-covid-vaccine/test.json", lines=True)
sub = pd.read_csv("./stanford-covid-vaccine/sample_submission.csv")

# # Filter training data
train = train[train.signal_to_noise > 1].reset_index(drop=True)

# Separate public and private test data based on sequence length
test_pub = test[test["seq_length"] == 107].reset_index(drop=True)
test_pri = test[test["seq_length"] == 130].reset_index(drop=True)


# Load BPP (Base Pair Probability) matrices
def load_bpps(df, path="./stanford-covid-vaccine/bpps/"):
    """Loads BPP matrices for a given DataFrame."""
    bpps = []
    for id_val in tqdm(df["id"], desc="Loading BPPs"):
        bpps.append(np.load(f"{path}{id_val}.npy"))
    return np.array(bpps)


As = load_bpps(train)
As_pub = load_bpps(test_pub)
As_pri = load_bpps(test_pri)

# Get target column names from sample submission
targets = list(sub.columns[1:])

# Prepare y_train (target values for training)
seq_len_train_example = train["seq_length"].iloc[0]
seq_len_target_example = train["seq_scored"].iloc[0]
ignore_value = -10000
ignore_length = seq_len_train_example - seq_len_target_example

y_train = []
for target in targets:
    y = np.vstack(train[target])
    # Pad with ignore_value for positions not scored
    dummy = np.zeros([y.shape[0], ignore_length]) + ignore_value
    y = np.hstack([y, dummy])
    y_train.append(y)
y = np.stack(y_train, axis=2)

Loading BPPs:   0%|          | 0/2096 [00:00<?, ?it/s]

Loading BPPs:   0%|          | 0/629 [00:00<?, ?it/s]

Loading BPPs:   0%|          | 0/3005 [00:00<?, ?it/s]

# Adjacency Matrix Preprocessing

In [6]:
def get_structure_adj(df):
    Ss = []
    for i in tqdm(range(len(df)), desc="Getting Structure Adjacency"):
        seq_length = df["seq_length"].iloc[i]
        structure = df["structure"].iloc[i]
        sequence = df["sequence"].iloc[i]

        cue = []
        # Separate adjacency matrices for different base pairs, then sum them up
        a_structures = {
            ("A", "U"): np.zeros([seq_length, seq_length]),
            ("C", "G"): np.zeros([seq_length, seq_length]),
            ("U", "G"): np.zeros([seq_length, seq_length]),
            ("U", "A"): np.zeros([seq_length, seq_length]),
            ("G", "C"): np.zeros([seq_length, seq_length]),
            ("G", "U"): np.zeros([seq_length, seq_length]),
        }
        
        for j in range(seq_length):
            if structure[j] == "(":
                cue.append(j)
            elif structure[j] == ")":
                if cue:
                    start = cue.pop()
                    a_structures[(sequence[start], sequence[j])][start, j] = 1
                    a_structures[(sequence[j], sequence[start])][j, start] = 1
        
        # Sum all specific base pair adjacency matrices into a single one
        a_strc = np.stack([a for a in a_structures.values()], axis=2)
        a_strc = np.sum(a_strc, axis=2, keepdims=True)
        Ss.append(a_strc)
    
    return np.array(Ss)


Ss = get_structure_adj(train)
Ss_pub = get_structure_adj(test_pub)
Ss_pri = get_structure_adj(test_pri)


def get_distance_matrix(bpps_array):
    seq_length = bpps_array.shape[1]
    idx = np.arange(seq_length)
    Ds = []
    for i in range(len(idx)):
        d = np.abs(idx[i] - idx)
        Ds.append(d)

    Ds = np.array(Ds) + 1
    Ds = 1 / Ds
    Ds = Ds[None, :, :]
    Ds = np.repeat(Ds, len(bpps_array), axis=0)
    
    Dss = []
    for i in [1, 2, 4]:
        Dss.append(Ds ** i)
    Ds = np.stack(Dss, axis=3)
    return Ds

Ds = get_distance_matrix(As)
Ds_pub = get_distance_matrix(As_pub)
Ds_pri = get_distance_matrix(As_pri)

# Concatenate all adjacency features: BPPs, Structure Adjacency, Distance Adjacency
As = np.concatenate([As[:, :, :, None], Ss, Ds], axis=3).astype(np.float32)
As_pub = np.concatenate([As_pub[:, :, :, None], Ss_pub, Ds_pub], axis=3).astype(np.float32)
As_pri = np.concatenate([As_pri[:, :, :, None], Ss_pri, Ds_pri], axis=3).astype(np.float32)

# Free up memory
del Ss, Ds, Ss_pub, Ds_pub, Ss_pri, Ds_pri
gc.collect()

Getting Structure Adjacency:   0%|          | 0/2096 [00:00<?, ?it/s]

Getting Structure Adjacency:   0%|          | 0/629 [00:00<?, ?it/s]

Getting Structure Adjacency:   0%|          | 0/3005 [00:00<?, ?it/s]

109

# Node Feature Preprocessing

In [7]:
def return_ohe(n, i):
    tmp = [0] * n
    tmp[i] = 1
    return tmp


def get_input(df):
    # One-hot encode sequence (bases)
    mapping_node = {s: return_ohe(4, i) for i, s in enumerate(["A", "G", "C", "U"])}
    X_node = np.stack(df["sequence"].apply(lambda x: list(map(lambda y: mapping_node[y], list(x)))))

    # One-hot encode predicted loop type
    mapping_loop = {s: return_ohe(7, i) for i, s in enumerate(["S", "M", "I", "B", "H", "E", "X"])}
    X_loop = np.stack(df["predicted_loop_type"].apply(lambda x: list(map(lambda y: mapping_loop[y], list(x)))))
    
    X_node = np.concatenate([X_node, X_loop], axis=2)
    
    # Interaction features: create a unique integer for each combination of features
    # and then one-hot encode these combined features.
    a = np.sum(X_node * (2 ** np.arange(X_node.shape[2])[None, None, :]), axis=2)
    vocab = sorted(set(a.flatten()))
    ohes = []
    for v in vocab:
        ohes.append(a == v)
    ohes = np.stack(ohes, axis=2)
    
    X_node = np.concatenate([X_node, ohes], axis=2).astype(np.float32)
    
    return X_node


X_node = get_input(train)
X_node_pub = get_input(test_pub)
X_node_pri = get_input(test_pri)

# Pre-training the Denoising Autoencoder

In [ ]:
# Dynamically get input shapes for model construction
base_model_ae = get_base(X_node.shape[2], As.shape[3])
ae_model = get_ae_model(base_model_ae, X_node.shape[2])

# Train denoising autoencoder using all data (train, public test, private test)
dummy_ae_target = np.zeros_like(X_node[:, :, 0])
dummy_ae_target_pub = np.zeros_like(X_node_pub[:, :, 0])
dummy_ae_target_pri = np.zeros_like(X_node_pri[:, :, 0])

for i in range(20 // 5):
    print(f"Pre-training Iteration {i+1}/{20 // 5}")
    print("Training on main dataset")
    ae_model.fit([X_node, As], [dummy_ae_target],
                    epochs=5,
                    batch_size=32,
                    verbose=1)

    print("Training on public test dataset")
    ae_model.fit([X_node_pub, As_pub], [dummy_ae_target_pub],
                    epochs=5,
                    batch_size=32,
                    verbose=1)

    print("Training on private test dataset")
    ae_model.fit([X_node_pri, As_pri], [dummy_ae_target_pri],
                    epochs=5,
                    batch_size=32,
                    verbose=1)
    gc.collect()
base_model_ae.save_weights("./base_ae.weights.h5")

Pre-training Iteration 1/4
Training on main dataset
Epoch 1/5
66/66 [==============================] - 36s 142ms/step - loss: 0.8666
Epoch 2/5
66/66 [==============================] - 7s 102ms/step - loss: 0.2422
Epoch 3/5
66/66 [==============================] - 7s 102ms/step - loss: 0.1132
Epoch 4/5
66/66 [==============================] - 7s 107ms/step - loss: 0.0717
Epoch 5/5
66/66 [==============================] - 7s 105ms/step - loss: 0.0541
Training on public test dataset
Epoch 1/5
20/20 [==============================] - 4s 210ms/step - loss: 0.0459
Epoch 2/5
20/20 [==============================] - 2s 113ms/step - loss: 0.0436
Epoch 3/5
20/20 [==============================] - 2s 106ms/step - loss: 0.0403
Epoch 4/5
20/20 [==============================] - 2s 102ms/step - loss: 0.0353
Epoch 5/5
20/20 [==============================] - 2s 100ms/step - loss: 0.0320
Training on private test dataset
Epoch 1/5
94/94 [==============================] - 26s 127ms/step - loss: 0.0362
E

# Main Training (Regression) with KFold Cross-Validation

In [9]:
kfold = KFold(5, shuffle=True, random_state=42)

scores = []
# Initialize prediction arrays with appropriate shapes
# For train data predictions (oof - Out Of Fold predictions)
preds_oof = np.zeros([len(X_node), X_node.shape[1], len(targets)])
# For public test data predictions
p_pub_total = np.zeros([len(X_node_pub), X_node_pub.shape[1], len(targets)])
# For private test data predictions
p_pri_total = np.zeros([len(X_node_pri), X_node_pri.shape[1], len(targets)])

for fold, (tr_idx, va_idx) in enumerate(kfold.split(X_node, As)):
    print(f"Fold {fold} Start")
    
    # Split data for current fold
    X_node_tr, X_node_va = X_node[tr_idx], X_node[va_idx]
    As_tr, As_va = As[tr_idx], As[va_idx]
    y_tr, y_va = y[tr_idx], y[va_idx]
    
    # Initialize base model for current fold
    base_model_fold = get_base(X_node.shape[2], As.shape[3])

    # Load pre-trained weights
    base_model_fold.load_weights("./base_ae.weights.h5")
        
    # Get the full regression model
    model = get_model(base_model_fold, seq_len_target_example, As.shape[3])

    # Train the regression model with progressive batch sizes and epochs
    for epochs, batch_size in zip([30, 10, 3, 3, 5, 5], [8, 16, 32, 64, 128, 256]):
        print(f"Epochs: {epochs}, Batch size: {batch_size}")
        if batch_size >= 128:
            # Use CPU to avoid GPU memory issues
            with tf.device('/CPU:0'):
                history = model.fit([X_node_tr, As_tr], [y_tr],
                                    validation_data=([X_node_va, As_va], [y_va]),
                                    epochs=epochs,
                                    batch_size=batch_size,
                                    validation_freq=3,
                                    verbose=1)
        else:
            history = model.fit([X_node_tr, As_tr], [y_tr],
                                validation_data=([X_node_va, As_va], [y_va]),
                                epochs=epochs,
                                batch_size=batch_size,
                                validation_freq=3,
                                verbose=1)
        
    # Save model weights after training for the current fold
    model.save_weights(f"./model{fold}.weights.h5")
    
    # Make out-of-fold predictions
    p_fold_val = model.predict([X_node_va, As_va])
    fold_mcrmse = mcrmse(y_va, p_fold_val, seq_len_target_example)
    scores.append(fold_mcrmse)
    print(f"Fold {fold}: MCRMSE {scores[-1]}")
    
    preds_oof[va_idx] = p_fold_val
    
    # Predict on public and private test sets for ensembling
    p_pub_total += model.predict([X_node_pub, As_pub])
    p_pri_total += model.predict([X_node_pri, As_pri])
    
    del X_node_tr, X_node_va, As_tr, As_va, y_tr, y_va, base_model_fold, model
    gc.collect()

# Average predictions from all folds
p_pub_total /= kfold.n_splits
p_pri_total /= kfold.n_splits

# Save Out-Of-Fold predictions
pd.to_pickle(preds_oof, "oof.pkl")

print("\nCross-Validation Results")
print("Individual Fold MCRMSE Scores:", scores)
print("Mean MCRMSE across all folds:", np.mean(scores))

Fold 0 Start
Epochs: 30, Batch size: 8
Epoch 1/30
210/210 [==============================] - 44s 99ms/step - loss: 0.4362
Epoch 2/30
210/210 [==============================] - 18s 87ms/step - loss: 0.3447
Epoch 3/30
210/210 [==============================] - 22s 107ms/step - loss: 0.3267 - val_loss: 0.3022
Epoch 4/30
210/210 [==============================] - 19s 89ms/step - loss: 0.3125
Epoch 5/30
210/210 [==============================] - 19s 88ms/step - loss: 0.3014
Epoch 6/30
210/210 [==============================] - 20s 98ms/step - loss: 0.2934 - val_loss: 0.2739
Epoch 7/30
210/210 [==============================] - 18s 87ms/step - loss: 0.2853
Epoch 8/30
210/210 [==============================] - 20s 95ms/step - loss: 0.2817
Epoch 9/30
210/210 [==============================] - 21s 100ms/step - loss: 0.2757 - val_loss: 0.2613
Epoch 10/30
210/210 [==============================] - 19s 91ms/step - loss: 0.2734
Epoch 11/30
210/210 [==============================] - 19s 90ms/step - 

# Generate Submission File

In [10]:
preds_ls = []
# Process public test set predictions
for i, uid in enumerate(test_pub.id):
    single_pred = p_pub_total[i]
    single_df = pd.DataFrame(single_pred[:, :test_pub["seq_scored"].iloc[i]], columns=targets)
    single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]
    preds_ls.append(single_df)

# Process private test set predictions
for i, uid in enumerate(test_pri.id):
    single_pred = p_pri_total[i]
    single_df = pd.DataFrame(single_pred[:, :test_pri["seq_scored"].iloc[i]], columns=targets)
    single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]
    preds_ls.append(single_df)

# Concatenate all predictions and save to submission.csv
preds_df = pd.concat(preds_ls)
preds_df.to_csv("submission.csv", index=False)

preds_df.head()

,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C,id_seqpos
0,0.791276,0.666923,1.715345,0.549008,0.724414,id_00073f8be_0
1,2.432134,3.419400,4.453869,3.341697,2.800921,id_00073f8be_1
2,1.735907,0.774205,0.850415,0.850344,0.801502,id_00073f8be_2
3,1.351261,1.151601,1.282416,1.576227,1.572157,id_00073f8be_3
4,0.841960,0.641961,0.657673,0.859995,0.873872,id_00073f8be_4
